In [2]:
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings("ignore")
import spacy
import re
import string
import pickle

In [76]:
reviews ={"reviews":[ " I can't believe I wasted my money on this keyboard and mouse",
            "I really love this device",
            "It's quite good, but not so great. They can do better",
            "I hate this device, terrible!!!",
            "Where the hell is my refund for this damn product (mouse)",
            "I believe Ebay is so much better than the crap that is HERE!!!",
            "worst keyboard ever",
            "distasteful to the fullest!!!",
            "spoilt on the first day",
            "never shopping here again, waste of money"]}

In [77]:
# https://stackoverflow.com/questions/12851791/removing-numbers-from-string
def preprocess(sent):
    '''Cleans text data up, leaving only 2 or
        more char long non-stepwords composed of A-Z & a-z only
        in lowercase'''
    # lowercase
    sentence = sent.lower()

    # Remove RT
    sentence = re.sub('RT @\w+: '," ",sentence)

    # Remove special characters
    sentence = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ", sentence)

    # Removing digits
    sentence = sentence.translate(str.maketrans('', '', string.digits))

    # Removing puntuactions
    # sentence = sentence.translate(str.maketrans('', '', string.punctuation))

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)  
    # When we remove apostrophe from the word "Mark's", 
    # the apostrophe is replaced by an empty space. 
    # Hence, we are left with single character "s" that we are removing here.

    # Remove multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)  
    # Next, we remove all the single characters and replace it by a space 
    # which creates multiple spaces in our text. 
    # Finally, we remove the multiple spaces from our text as well.

    return sentence


In [78]:
reviews = pd.DataFrame(reviews)

In [79]:
reviews

,reviews
0,I can't believe I wasted my money on this key...
1,I really love this device
2,"It's quite good, but not so great. They can do..."
3,"I hate this device, terrible!!!"
4,Where the hell is my refund for this damn prod...
5,I believe Ebay is so much better than the crap...
6,worst keyboard ever
7,distasteful to the fullest!!!
8,spoilt on the first day
9,"never shopping here again, waste of money"


In [80]:
from spacy.lang.en.stop_words import STOP_WORDS

# print(STOP_WORDS,'\n') # <- set of Spacy's default stop words
all_stopwords = STOP_WORDS
# Adding several stopwords
# all_stopwords |= {'not',"no", "n't", 'n’t','n‘t','cannot','none','without','against'}

# Removing several stop words
all_stopwords-= {'not',"no", "n't", 'n’t','n‘t','cannot','none','without','against','off','against','too'}
my_stop_words = set(all_stopwords) # My own stop words

In [81]:
reviews["cleaned"] = reviews["reviews"].apply(preprocess)

In [82]:
reviews

,reviews,cleaned
0,I can't believe I wasted my money on this key...,can believe wasted my money on this keyboard ...
1,I really love this device,i really love this device
2,"It's quite good, but not so great. They can do...",it quite good but not so great they can do better
3,"I hate this device, terrible!!!",i hate this device terrible
4,Where the hell is my refund for this damn prod...,where the hell is my refund for this damn prod...
5,I believe Ebay is so much better than the crap...,i believe ebay is so much better than the crap...
6,worst keyboard ever,worst keyboard ever
7,distasteful to the fullest!!!,distasteful to the fullest
8,spoilt on the first day,spoilt on the first day
9,"never shopping here again, waste of money",never shopping here again waste of money


In [83]:
nlp = spacy.load("en_core_web_sm")
def spacy_tokeniser(sent):
    sent = sent.strip().lower()
    doc = nlp(sent)
    mytokens = [token.lemma_ for token in doc if token.text not in my_stop_words]
    return mytokens

In [84]:
reviews['tokens']= reviews['cleaned'].apply(spacy_tokeniser)

In [85]:
reviews

,reviews,cleaned,tokens
0,I can't believe I wasted my money on this key...,can believe wasted my money on this keyboard ...,"[believe, waste, money, keyboard, mouse]"
1,I really love this device,i really love this device,"[love, device]"
2,"It's quite good, but not so great. They can do...",it quite good but not so great they can do better,"[good, not, great, well]"
3,"I hate this device, terrible!!!",i hate this device terrible,"[hate, device, terrible]"
4,Where the hell is my refund for this damn prod...,where the hell is my refund for this damn prod...,"[hell, refund, damn, product, mouse]"
5,I believe Ebay is so much better than the crap...,i believe ebay is so much better than the crap...,"[believe, ebay, well, crap]"
6,worst keyboard ever,worst keyboard ever,"[bad, keyboard]"
7,distasteful to the fullest!!!,distasteful to the fullest,"[distasteful, full]"
8,spoilt on the first day,spoilt on the first day,"[spoilt, day]"
9,"never shopping here again, waste of money",never shopping here again waste of money,"[shop, waste, money]"


In [86]:
vectorizer = pickle.load(open('../../../../../../word2vec_files/glove_twitter_200.pkl', "rb"))
model = pickle.load(open('hgb_200.pkl','rb'))

In [87]:
# Vecctoriser of Word2vec
def sent_vec(sent):
    vector_size = vectorizer.vector_size
    w2v_resolution = np.zeros(vector_size)
    # print(w2v_resolution)
    ctr = 1
    for w in sent:
        if w in vectorizer:
            ctr += 1
            w2v_resolution += vectorizer[w]
    w2v_resolution = w2v_resolution/ctr
    # print(w2v_resolution)
    return w2v_resolution

In [88]:
reviews["vectors"] = reviews['tokens'].apply(sent_vec)

In [89]:
X = reviews['vectors'].tolist()

In [90]:
y_pred = model.predict(X)

In [91]:
reviews["predictions"] = y_pred.tolist()
reviews

,reviews,cleaned,tokens,vectors,predictions
0,I can't believe I wasted my money on this key...,can believe wasted my money on this keyboard ...,"[believe, waste, money, keyboard, mouse]","[0.005858665332198143, 0.18548966757953167, 0....",0
1,I really love this device,i really love this device,"[love, device]","[-0.1616696628431479, 0.06851699699958165, -0....",1
2,"It's quite good, but not so great. They can do...",it quite good but not so great they can do better,"[good, not, great, well]","[0.030389800295233728, 0.02629140168428421, 0....",1
3,"I hate this device, terrible!!!",i hate this device terrible,"[hate, device, terrible]","[0.10614324826747179, 0.06222624983638525, -0....",-1
4,Where the hell is my refund for this damn prod...,where the hell is my refund for this damn prod...,"[hell, refund, damn, product, mouse]","[0.15782649939258894, 0.1072516633818547, -0.1...",0
5,I believe Ebay is so much better than the crap...,i believe ebay is so much better than the crap...,"[believe, ebay, well, crap]","[-0.009377998113632203, 0.1232600063085556, -0...",0
6,worst keyboard ever,worst keyboard ever,"[bad, keyboard]","[0.10655266791582108, -0.0725806641081969, 0.0...",-1
7,distasteful to the fullest!!!,distasteful to the fullest,"[distasteful, full]","[-0.10743000109990437, -0.027565665543079376, ...",0
8,spoilt on the first day,spoilt on the first day,"[spoilt, day]","[0.019213999932010967, 0.009006659189860025, -...",0
9,"never shopping here again, waste of money",never shopping here again waste of money,"[shop, waste, money]","[-0.20904649887233973, -0.03836749866604805, 0...",0
